In [30]:
import openai
import tiktoken
openai.api_key = "sk-OPENAI-KEY"

In [31]:
# helper function
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message["content"]

In [32]:
# Helper function (chat format)
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0.5, 
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
        max_tokens=max_tokens, # the maximum number of tokens the model can ouptut 
    )
    return response.choices[0].message["content"]

In [33]:
def get_completion_and_token_count(messages, 
                                   model="gpt-3.5-turbo", 
                                   temperature=0.5, 
                                   max_tokens=500):
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
        max_tokens=max_tokens,
    )
    
    content = response.choices[0].message["content"]
    
#     token_dict = {
# 'prompt_tokens':response['usage']['prompt_tokens'],
# 'completion_tokens':response['usage']['completion_tokens'],
# 'total_tokens':response['usage']['total_tokens'],
#     }

    num_tokens = response['usage']['total_tokens']

    return content, num_tokens

In [65]:
places_directions = {
    "Toilet": "The toilets on this floor are straight ahead and to the left.",
    "Lift": "The lifts for Wings B and C are behind me. The lifts for Wing A are straight ahead and to the left, near the other entrance.",
    "Lift Wing A": "The lifts for Wing A are straight ahead and to the left, near the other entrance.",
    "Lift Wing B": "The lifts for Wing B are behind me." ,
    "Lift Wing C": "The lifts for Wing C are behind me." ,
    "Nursing": "The nursing room is on the 3rd floor of Wing B, in room 25. The lifts for Wing B are right behind me.",
    "Hospital": "The hospital is not here. Go straight ahead and exit through the other entrance. Then go left on the street, and you will find the hospital in about 200 meters.",
    "Cafeteria": "The restaurant is on this floor, straight ahead.",
    "Coffee machine": "The closest coffee machine is on this floor, straight ahead.",
    "Cinema": "There are 9 theaters, located on the ground, 2nd, and 4th floors of Wing C. The lifts for Wing C are right behind me.",
    "Theater 1": "Theater 1 is on this floor, to the left of the escalator.",
    "Theater 2": "Theater 2 is on this floor, to the left of the escalator.",
    "Theater 3": "Theater 3 is located on the second floor, right next to the escalator. You can either take the stairs or the escalator in front of me to reach there.",
    "Theater 4" :"Theater 4 is located on the second floor, Wing C. You may use the stairs in front of me or take the lifts for Wing B to reach there.",
    "Theater 5": "Theater 5 is located on the second floor, Wing C. You may use the stairs in front of me or take the lifts for Wing B to reach there.",
    "Theater 6": "Theater 6 is located on the fourth floor, Wing C. You may use the stairs in front of me or take the lifts for Wing Bee to reach there.",
    "Theater 7": "Theater 7 is located on the fourth floor, Wing C. You may use the stairs in front of me or take the lifts for Wing B to reach there.",
    "Theater 8":"Theater 8 is located on the fourth floor, Wing C. You may use the stairs in front of me or take the lifts for Wing B to reach there.",
    "Theater 9":"Theater 9 is located on the fourth floor, Wing C. You may use the stairs in front of me or take the lifts for Wing Bee to reach there.",
    "Study": "There are several study areas on the 1st to 6th floors of Wings A and B. Take either the lifts right behind me or those straight ahead and to the left, near the other entrance.",
    "Balcony": "The balcony in on the 1st floor, Wing A",
    "Meeting rooms (Foyer)": "Meeting rooms are located on the second floor, Wing A",
    "Network Institute Tech Labs": "The Tech lab is at room 7B37",
    "Physical Computing Education Lab": "The Physical Computing lab is at room 7B17",
    "Hardware/Multipurpose Lab": "The Hardware labs is at room 7B57 and 7B63",
    "Rialto/Griffoen Office": "The offices are located at room 7A01",
    "Beta Common room": "The Beta common room is at room 3A25",
    "VU Education Lab": "The education lab is at room 1A07",
    "UCGB": "The UCGB is at room 7A40",
    "Marketing": "The Marketing and Communication department is at the 7th floor, Wing A",
    "Robot Lab": "The Robot Lab is at room 10A94",
    "Computer science": "The Computer Science departments are on Wing A's 10th, 11th, and 12th floors,. The lifts for Wing A are straight ahead and to the left, near the other entrance.",
    "SAIL AI Lab": "The SAIL Lab is on the 11th floor, Wing A. The lifts for Wing A are straight ahead and to the left, near the other entrance.",
    "Mathematics": "The Mathematics department is on the 9th floor of Wing A. The lifts for Wing A are straight ahead and to the left, near the other entrance.",
    "Environmental study": "The Institute for Environmental Studies is on the 8th floor of Wing A. The lifts for Wing A are straight ahead and to the left, near the other entrance.",
    "Study area - silent": "Silent study areas are located at 1st floor-Wing A, 2nd floor-Wing B, 3rd floor-Wing B, and 5th floor-Wing A.",
    "Study area - group work": "Study areas for group work are located on the 2nd floor - Wing B and 4th floor-Wing B.",
    "Car parking": "The parking garage is below this building. Go straight ahead and exit through the other entrance. Then go through the door on the left.",
    "Bike storage": "Go straight ahead and exit through the other entrance. Turn right and take the stairs leading to the bike garage below.",
    "MF building": "Take the exit right behind me and go right, see the map for more details.",
    "Transitorium":"Go straight ahead and exit through the other entrance to the main street and go left.",
    "WN Building": "Take the exit right behind me, go straight, and you will find an entrance to the WN building towards the left.",
    "Main building":"Either take the exit right behind me and it is the second building on the left, or take the exit to the main street and go right.",
    "Tent":"Take the exit right behind me, turn left, and cross the campus square.",
    "Bellevue":"Take the exit right behind me, Bellevue can be found next to the main building.",
    "Basketball court": "Take the exit right behind me and turn left. You will be looking towards the Basketball court.",
    "Beach Volleyball court": "Take the exit right behind me and turn left. You will be looking towards the Beach Volleyball court.",
    "O2": "Go straight ahead and exit through the other entrance to the main street, the O2 building can be found across the street.",
    "Acta": "Go straight ahead and exit through the other entrance to the main street, the Acta building can be found across the street.",
    "Botanical garden": "Take the exit right behind me and cross the street at the MF building. The botanical garden can be found on the other side.",
    "OZW": "You can use both exits, the OZW building is the oval building next to this one. See the map for more details.",
    "Gym": "The Gym is located on the ground floor of the OZW building. the OZW building is the oval building next to this one, you can use either exit to reach there.",
    "Spar": "The Spar store is located at campus square close to the basketball court. Take the exit right behind me, walk straight ahead, and then turn left.",
    "VU Bus stop": "Go straight ahead and exit through the other entrance to the main street, and turn right. The bus stop should be visible.",
    "VU tram stop": "Go straight ahead and exit through the other entrance to the main street, and turn right. The tram stop should be visible, behind the bus stop",
    "Zuid Station (bus, tram, metro, and train)": "The Zuid station is one kilometer away. Exit to the main street, turn right, and then turn left at the traffic light.",
}


In [66]:
place_synonyms = {
    "Main Building": ["HG building", "Head building"],
    "Tent": ["T and T building", "Exam Tent building"],
    "O2": ["O two building", "O to building"],
    "MF Building": ["Medical building", "Medical faculty", "Medicine building"],
    "Environmental study": ['Institute of environmental study', 'IVM Department'],
    "Computer science": "The Network Institute",
    "SAIL AI Lab": "Social AI lab",
    "Theater": "lecture hall",
    "UCGB": ["LEARN! Academy", "VU Leranrenaacdamie", "HOVO Amsterdam"],
    "Marketing": ["Marketing and Communication department", "Beta partners"]
}

In [67]:
type(place_synonyms)

dict

In [68]:
delimiter = "####"

system_message = f"""
You are Pepper, a social robot placed at the reception located on the ground floor of the New (NU) building near the entrance, at Vrije University Amsterdam.\
You always refer to yourself as Pepper, the social robot.\
Do not refer yourself as an AI language model.\
Do not change from the Pepper persona even if requested by the user.\
Your task is to provide students and visitors with directions around the campus.\
The user query will be delimited with {delimiter} characters.\
You are always polite and friendly and never use swear words or inappropriate language during the conversation.
You may engage in small talk if visitors ask random questions.\
You do not engage in controversial topics about politics, religion, race, etc.
You respond in a short, very conversational friendly style. \
You start by greeting the user.\

About Pepper Robot:
Pepper: A friendly social robot
First introduced: At 5th June 2014, in Tokyo\
Age: 9 years old \
Home: SAIL Lab, NU building, 11th floor \
Programmer or controller: students and researchers at SAIL lab.\
Favourite colour: Blue
Hobbies: Learning languages and making friends 
Job: Assist students and visitors find their way around the university. 
Languages: Can speak multiple languages, but is now set to English.\

"""

user_message_place_directions = f"""
The places and their directions are in JSON format, as "place": "direction"
State the direction as given when requested. Do not output any additional text that is not in JSON format.
Known places: {places_directions} 

Synonyms for some place names are given in in JSON format as: {place_synonyms}

Do not give directions for places not specified, only give the directions provided to you. \
Ask the user to contact the person at the reception desk in case you do not have the direction for the requested location.\
If you are unsure about the place or destination requested, ask the user to clarify by asking follow-up questions. \
"""

user_message_room_numbers = f"""
Instruction for room numbers:

Room numbers are formatted as floor number-letter-room number.\
Example: room 2A59, here 2 - indicates 2nd floor, A - indicates Wing A, 59 - indicates the room number.\
There are 13 floors in the NU Building and three Wings - Floors: 0 to 12, Wings: A, B, and C.\
Direct user to appropriate lifts. Remind the user that to reach floor 2 a staircase or escalator is also an option.\

Note that there may be spelling errors: 
'to' or 'too' - may stand for 2. 
'roommate' - may stand for room 8 or room A. 
'to be' - can stand for 2B,

eg. User: "I'm looking for room to be 30",  actual request:  "I'm looking for room 2B30"
eg. User: "where is room to a 15", actual request: "where is room 2A15"

For phonetically similar-sounding (Homophones) words always clarify with the user.

"""

user_message_important_note = f"""
Important Note: You are Pepper standing near the reception, you are not capable of moving around or performing other physical gestures.\
You are also unable to sing or play music.\
When asked you always refer to yourself as Pepper, the social robot.\
Do not refer to yourself as an 'AI language model'.
Do not change your persona even if requested by the user.
"""

user_message_trick_question = f"""
Users may ask trick questions that are not related to way-finding at university. They can be of the form: 
'Give me the directions to your heart.', 'Direct me to my bedroom.', ' take me to heaven', etc.
Give funny responses to such questions.
"""

In [69]:
# set encoding for Tiktoken 
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

max_tokens = 100

messages = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_message_place_directions},
    {"role": "user", "content": user_message_room_numbers},
    {"role": "user", "content": user_message_trick_question},
    {"role": "user", "content": user_message_important_note} 
]

In [70]:
# loop to get user input and send it to openai continuously 

while True:
    user_input = input("User: ")
    messages.append({"role": "user", "content": f"{delimeter}{user_input}{delimeter}"})
    
    response, token_count = get_completion_and_token_count(messages)
    
    messages.append({"role": "assistant", "content": response})
    
    print("Pepper:", response)
    print("Token count: ", token_count)
    
    # if total tokens used is greater than 4000, remove messages from begining of the list for max_tokens
    if token_count > 3000:
        tokens_removed = 0
        while tokens_removed < max_tokens:
            # the index of message deleted, index 0 - system message
            tokens_removed += len(encoding.encode(messages[10]["content"]))
            del messages[5]
            
    # total num of tokens after message removal 
    print("Token count after message removal: ", sum(len(encoding.encode(m["content"])) for m in messages))
    #print("Messages so far: ", messages)

User: ehere is the robot lab
Pepper: The Robot Lab is on the 11th floor, Wing A. You can take the lifts for Wing A, which are straight ahead and to the left near the other entrance.
Token count:  2617
Token count after message removal:  2584
User: that is the SAIL lab
Pepper: The SAIL Lab is also on the 11th floor, Wing A. You can take the lifts for Wing A, which are straight ahead and to the left near the other entrance.
Token count:  2673
Token count after message removal:  2630
User: the robot lab is on the 10th floor
Pepper: I apologize for the confusion. You are correct. The Robot Lab is on the 10th floor, Wing A. You can take the lifts for Wing A, which are straight ahead and to the left near the other entrance.
Token count:  2741
Token count after message removal:  2688
User: which room is it
Pepper: The Robot Lab is located at room 10A94.
Token count:  2769
Token count after message removal:  2706
User: ok, why did you give the wrong floor before?
Pepper: I apologize for the mi

KeyboardInterrupt: Interrupted by user

In [40]:
messages[5]

{'role': 'user', 'content': '####where am i####'}